In [ ]:
%pip install -q otter-grader

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# **Домашнее задание 2.** Метод K-ближайших соседей

<p style="color:red">  
<b>Предупреждение:</b> в данном задании разрешено использовать только NumPy, Pandas и библиотеки для визуализации! 
</p>

На семинаре мы познакомились с реализацией KNN в библиотеке scikit-learn, обучили модели для задач классификации и регрессии и оценили их качество на синтетических данных.

Следуя высказыванию одного удивительного человека:

> «What I cannot create, I do not understand.» — Richard Feynman

в этом задании мы реализуем алгоритм k-ближайших соседей на чистом NumPy. Приступим!

In [ ]:
from typing import Any, Literal, Self

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def convert_pandas_to_numpy(func: callable) -> callable:
    def wrapper(*args: tuple[Any], **kwargs: dict[Any]):
        args = tuple(arg.values if isinstance(arg, (pd.DataFrame, pd.Series)) else arg for arg in args)
        kwargs = {
            key: arg.values if isinstance(arg, (pd.DataFrame, pd.Series)) else arg
            for key, arg in kwargs.items()
        }  # fmt: skip

        return func(*args, **kwargs)

    return wrapper

### **Задание 0.** Предобработка данных

Прочитайте данные из файла `Predict_Calorie_Expenditure_Dataset.csv`, при необходимости удалите пропущенные значения, проведите разведочный анализ данных.  

_Данное задание не оценивается, но его выполнение необходимо для получения полного балла._

_Points:_ 0

In [ ]:
data = ...
data = data.sample(5_000, random_state=42)

print(f"{data.shape = }")
data.head()

In [ ]:
grader.check("Task0")

Разобьем данные на обучающую, валидационную и тестовые подвыборки

In [ ]:
X, y = data.drop(columns="Calories"), data["Calories"]

X_test = X.sample(frac=0.2, random_state=42)
test_idx = X_test.index
y_test = y.loc[test_idx]

train_idx = X.index.difference(test_idx)
X_train = X.loc[train_idx]
y_train = y.loc[train_idx]

print(f"Размер обучающей выборки:\t{y_train.size} объектов")
print(f"Размер тестовой выборки:\t{y_test.size} объектов")

### **Задание 1.** Евклидово расстояние

<p style="color:red">  
<b>Предупреждение:</b> использование циклов <code>for</code> и <code>while</code> в данном задании запрещено!  
</p>  

Пусть матрица $ X_A \in \mathbb{R}^{n \times d} $ содержит объекты с известными целевыми переменными, а матрица $ X_B \in \mathbb{R}^{m \times d} $ — объекты, для которых мы хотим сделать предсказания. Для применения модели KNN нам необходимо научиться считать матрицу расстояний $ D \in \mathbb{R}^{n \times m} $.

В теории мы могли бы, используя вложенные циклы, пройти по строкам первой и второй матриц, считая расстояния между ними, но ярко-красное предупреждение выше не рекомендует нам этого делать. Значит, будем искать другой способ, а для этого посмотрим внимательнее на определение евклидова расстояния:

$$
d(\mathbf{v}, \mathbf{u}) = \sqrt{\sum_{i=1}^{d} (v_i - u_i)^2}, \quad \mathbf{v},\, \mathbf{u} \in \mathbb{R}^d.
$$

Теперь избавимся от корня:

$$
\begin{aligned}
d^2(\mathbf{v}, \mathbf{u})
&= \sum_{i=1}^{d} (v_i - u_i)^2 \\
&= \sum_{i=1}^{d} v_i^2 + \sum_{i=1}^{d} u_i^2 - 2 \sum_{i=1}^{d} v_i u_i \\
&= \lVert \mathbf{v} \rVert^2 + \lVert \mathbf{u} \rVert^2 - 2\,\mathbf{v}\cdot\mathbf{u}.
\end{aligned}
$$

Иными словами, для расчета евклидова расстояния в матричной форме нам всего-то и нужно, что посчитать нормы векторов в исходных матрицах и найти матрицу скалярных произведений. С этим уже можно работать!

**Подсказка:** рассмотрите произведение $ X_A X_B^\top $.

Реализуйте функцию `cdist`, которая вычисляет евклидовы расстояния между строками двух матриц.

_Points:_ 5

In [ ]:
@convert_pandas_to_numpy
def cdist(XA: np.ndarray, XB: np.ndarray) -> np.ndarray:
    """Вычисляет матрицу евклидовых расстояний между векторами из двух входных матриц.

    Args:
        XA (np.ndarray): Матрица размера (n, d), содержащая n d-мерных векторов.
        XB (np.ndarray): Матрица размера (m, d), содержащая m d-мерных векторов.

    Returns:
        np.ndarray: Матрица расстояний размера (n, m).
    """
    ...

In [ ]:
grader.check("Task1")

### **Задание 2.** Базовый класс

Наконец, мы готовы приступить к имплементации KNN. Сначала создадим базовый класс, который будет содержать логику обучения и поиска ближайших соседей.

_Points:_ 5

In [ ]:
class KNeighborsModel:
    def __init__(self, n_neighbors: int = 5, weights: Literal["uniform", "distance"] = "uniform") -> None:
        """
        Args:
            n_neighbors (int): Количество ближайших соседей K.
            weights (Literal["uniform", "distance"]): Метод агрегации целевых переменных соседей.
                По умолчанию равен uniform.
        """
        self.n_neighbors = n_neighbors
        self.weights = weights

    @convert_pandas_to_numpy
    def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
        """Обучает модель.

        Args:
            X (np.ndarray): Матрица размера (n, d), где n - количество объектов, d - количество признаков.
            y (np.ndarray): Вектор целевых переменных размера (n,).

        Returns:
            Self: Обученная модель.
        """
        assert X.shape[0] == y.shape[0], "`X` и `y` должны иметь одинаковую длину."

        ...

        return self

    @convert_pandas_to_numpy
    def predict(self, X: np.ndarray) -> np.ndarray:
        neighbors, distances = self._find_neighbors(X)
        return self._aggregate(neighbors, distances)

    def _find_neighbors(self, X: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Находит K-ближайших соседей.

        Args:
            X (np.ndarray): Матрица размера (m, d), где m - количество объектов, d - количество признаков.

        Returns:
            tuple[np.ndarray, np.ndarray]: Кортеж (neighbors, distances), где
                neighbors (m, k) - целевые переменные ближайших соседей,
                distances (m, k) - расстояния до ближайших соседей.
        """
        ...

    def _aggregate(self, neighbors: np.ndarray, distances: np.ndarray) -> np.ndarray:
        if self.weights == "uniform":
            return self._aggregate_uniform(neighbors)
        else:
            weights = ...
            return self._aggregate_weighted(neighbors, weights)

    def _aggregate_uniform(self, neighbors: np.ndarray) -> np.ndarray:
        raise NotImplementedError

    def _aggregate_weighted(self, neighbors: np.ndarray, weights: np.ndarray) -> np.ndarray:
        raise NotImplementedError

In [ ]:
grader.check("Task2")

### **Задание 3**. Метод K-ближайших соседей для задачи регрессии

_Points:_ 2

In [ ]:
class KNeighborsRegressor(KNeighborsModel):
    def _aggregate_uniform(self, neighbors: np.ndarray) -> np.ndarray:
        ...

    def _aggregate_weighted(self, neighbors: np.ndarray, weights: np.ndarray) -> np.ndarray:
        ...

In [ ]:
grader.check("Task3")

### **Задание 4.** Метрики качества регрессии. Root Mean Squared Error (RMSE)

Для оценки качества модели будем использовать корень из среднеквадратичной ошибки (root mean squared error, RMSE):

$$
\text{RMSE}(\mathbf{y}, \hat{\mathbf{y}}) = \sqrt{\frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i) ^ 2}.
$$

_Points:_ 2

In [ ]:
def root_mean_squared_error(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    """Рассчитывает RMSE.

    Args:
        y_true (np.ndarray): Истинные значения целевой переменной.
        y_pred (np.ndarray): Предсказанные значения целевой переменной.

    Returns:
        np.ndarray: Значение метрики RMSE.
    """
    assert y_true.shape == y_pred.shape, "Размеры `y_true` и `y_pred` должны совпадать."
    ...

In [ ]:
grader.check("Task4")

### **Задание 5.** Обучение модели

Теперь обучите модель с параметром `n_neighbors=10` на числовых признаках и оцените ее качество

_Points:_ 2

In [ ]:
NUMERIC_FEATURES = ...

In [ ]:
reg = ...
y_pred = ...

print(f"RMSE:\t{root_mean_squared_error(y_test, y_pred):.3f}")

### **Задание 6.** Масштабирование признаков

Вспомните, почему так важно масштабировать признаки при использовании KNN. Затем реализуйте класс `StandardScaler` и примените его к нашим данным.

_Points:_ 2

In [ ]:
class StandardScaler:
    @convert_pandas_to_numpy
    def fit(self, X: np.ndarray) -> Self:
        ...

        return self

    @convert_pandas_to_numpy
    def transform(self, X: np.ndarray) -> np.ndarray:
        ...

    @convert_pandas_to_numpy
    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        return self.fit(X).transform(X)

In [ ]:
...

In [ ]:
reg = ...
y_pred = ...

print(f"RMSE:\t{root_mean_squared_error(y_test, y_pred):.3f}")

In [ ]:
grader.check("Task6")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)